In [1]:
import numpy as np
import random
import time
import heapq
import matplotlib.pyplot as plt

def generate_puzzle(n):
    numbers = list(range(n * n))
    random.shuffle(numbers)
    return np.array(numbers).reshape(n, n)

def goal_test(puzzle):
    n = puzzle.shape[0]
    goal = np.append(np.arange(1, n * n), 0)
    return np.array_equal(puzzle.flatten(), goal)

def find_zero(state):
    return tuple(np.argwhere(state == 0)[0])

def move_tile(puzzle, direction):
    row, col = find_zero(puzzle)
    new_puzzle = puzzle.copy()
    if direction == "up" and row > 0:
        new_puzzle[row, col], new_puzzle[row - 1, col] = new_puzzle[row - 1, col], new_puzzle[row, col]
    elif direction == "down" and row < puzzle.shape[0] - 1:
        new_puzzle[row, col], new_puzzle[row + 1, col] = new_puzzle[row + 1, col], new_puzzle[row, col]
    elif direction == "left" and col > 0:
        new_puzzle[row, col], new_puzzle[row, col - 1] = new_puzzle[row, col - 1], new_puzzle[row, col]
    elif direction == "right" and col < puzzle.shape[1] - 1:
        new_puzzle[row, col], new_puzzle[row, col + 1] = new_puzzle[row, col + 1], new_puzzle[row, col]
    else:
        return None
    return new_puzzle

def ucs_solve(puzzle):
    queue = [(0, puzzle)]
    visited = set()
    start_time = time.time()
    while queue:
        cost, current_state = heapq.heappop(queue)
        state_tuple = tuple(current_state.flatten())
        if state_tuple in visited:
            continue
        visited.add(state_tuple)
        if goal_test(current_state):
            return cost, time.time() - start_time
        for move in ["left", "right", "up", "down"]:
            new_state = move_tile(current_state, move)
            if new_state is not None and tuple(new_state.flatten()) not in visited:
                heapq.heappush(queue, (cost + 1, new_state))
        if time.time() - start_time > 60:
            return None, None
    return None, None

def bidirectional_solve(puzzle):
    start_queue = [puzzle]
    goal = np.append(np.arange(1, puzzle.size), 0).reshape(puzzle.shape)
    goal_queue = [goal]
    visited_start = set()
    visited_goal = set()
    start_time = time.time()
    while start_queue and goal_queue:
        current_start = start_queue.pop(0)
        state_tuple_start = tuple(current_start.flatten())
        if state_tuple_start in visited_start:
            continue
        visited_start.add(state_tuple_start)
        if state_tuple_start in visited_goal:
            return len(visited_start) + len(visited_goal), time.time() - start_time
        for move in ["left", "right", "up", "down"]:
            new_state = move_tile(current_start, move)
            if new_state is not None and tuple(new_state.flatten()) not in visited_start:
                start_queue.append(new_state)
        current_goal = goal_queue.pop(0)
        state_tuple_goal = tuple(current_goal.flatten())
        if state_tuple_goal in visited_goal:
            continue
        visited_goal.add(state_tuple_goal)
        if state_tuple_goal in visited_start:
            return len(visited_start) + len(visited_goal), time.time() - start_time
        for move in ["left", "right", "up", "down"]:
            new_state = move_tile(current_goal, move)
            if new_state is not None and tuple(new_state.flatten()) not in visited_goal:
                goal_queue.append(new_state)
        if time.time() - start_time > 60:
            return None, None
    return None, None

def main():
    n = int(input("Enter value of n: "))
    ucs_times = []
    bidirectional_times = []
    for _ in range(10):
        puzzle = generate_puzzle(n)
        ucs_cost, ucs_time = ucs_solve(puzzle)
        bidirectional_cost, bidirectional_time = bidirectional_solve(puzzle)
        if ucs_time is not None:
            ucs_times.append(ucs_time)
        if bidirectional_time is not None:
            bidirectional_times.append(bidirectional_time)
    avg_ucs_time = sum(ucs_times) / len(ucs_times) if ucs_times else None
    avg_bidirectional_time = sum(bidirectional_times) / len(bidirectional_times) if bidirectional_times else None
    plt.figure(figsize=(10, 5))
    plt.bar(["UCS", "Bidirectional"], [avg_ucs_time, avg_bidirectional_time], color=['blue', 'red'])
    plt.xlabel("Algorithm")
    plt.ylabel("Average Time (seconds)")
    plt.title(f"Comparison of UCS and Bidirectional Search for n={n}")
    plt.show()

if __name__ == "__main__":
    main()

Enter value of n:  3


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()